In [1]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from pathlib import Path
from haystack import Document
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

document_store = InMemoryDocumentStore()

c:\Users\Me\Desktop\chat-with-audio\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
doc_files = list(Path("./data").glob("**/*.txt"))
doc_files

[WindowsPath('data/large-data.txt')]

In [3]:
doc_content = doc_files[0].read_text()
doc_content

"In this video, I'm talking about how to get your first 100 customers for your SaaS product. I'm gonna be offering up actionable strategies and tactics on how you can get this done. I'm Rob Walling. I'm a startup founder with multiple exits, author of 3 books about building startups, and an investor in more than 100 companies. When most people think about building a product, they think about how can I build the product and then market it later? How can I build something and then go find people who have this need? And that's the opposite way to go about it. If I were launching a product today, I would say finding your first 100 customers starts way before you write that first line of code. The first thing you should do is look for advantages that you have, and you do this when you're choosing the idea, before you write a line of code, before you choose the idea, before you validate the idea. You think about, do I have an audience in any space that I could build a product for, or do I ha

In [4]:
docs = [Document(content=file.read_text()) for file in doc_files]
docs

[Document(id=c187895e69579e138b55b8908125b9274ba068fab93b8b189ae6a37e4418ea62, content: 'In this video, I'm talking about how to get your first 100 customers for your SaaS product. I'm gonn...')]

In [5]:
doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
doc_embedder.warm_up()

c:\Users\Me\Desktop\chat-with-audio\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.81it/s]


1

In [7]:
document_store

In [8]:
from haystack.components.builders import PromptBuilder

text_embedder = SentenceTransformersTextEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
retriever = InMemoryEmbeddingRetriever(document_store)

template = """
Given the following information, answer the question. Use the information from the documents to support your answer. Do not use any external information or make up any information. If you don't know the answer, write "I don't know".

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

In [9]:
prompt_builder.run(documents=docs, question="How to get my first saas customers?")

{'prompt': '\nGiven the following information, answer the question. Use the information from the documents to support your answer. Do not use any external information or make up any information. If you don\'t know the answer, write "I don\'t know".\n\nContext:\n\n    In this video, I\'m talking about how to get your first 100 customers for your SaaS product. I\'m gonna be offering up actionable strategies and tactics on how you can get this done. I\'m Rob Walling. I\'m a startup founder with multiple exits, author of 3 books about building startups, and an investor in more than 100 companies. When most people think about building a product, they think about how can I build the product and then market it later? How can I build something and then go find people who have this need? And that\'s the opposite way to go about it. If I were launching a product today, I would say finding your first 100 customers starts way before you write that first line of code. The first thing you should do 

In [10]:
import os
from getpass import getpass
from haystack.components.generators import OpenAIGenerator

generator = OpenAIGenerator(model="gpt-4o-mini")

In [11]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [12]:
question = "How to get my first saas customers?"

response = basic_rag_pipeline.run(
    {"text_embedder": {"text": question}, "prompt_builder": {"question": question}}
)

print(response["llm"]["replies"][0])

Batches: 100%|██████████| 1/1 [00:00<00:00, 108.41it/s]


To get your first SaaS customers, you should follow these actionable strategies:

1. **Start Before Coding**: Begin looking for customers and validating your idea before you write any code. Identify any advantages you may have, such as an existing audience or a network of connections.

2. **Build an Audience or Network**: If you lack an audience, focus on building a network within entrepreneur communities online, like indiehackers.com or microconfconnect.

3. **Create a Landing Page**: Set up a landing page that outlines your product idea and captures emails from interested people. This helps validate interest and allows for customer development.

4. **Engage with Your Early List**: Communicate with the people who sign up on your email list about your product's development. Gather feedback on your mockups and assess whether potential customers would be willing to pay for your product.

5. **Launch to Your Email List**: Once you’re ready, launch your product to your email list first, pr